<a href="https://colab.research.google.com/github/genie0320/langchain/blob/main/04_RAG_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# colab 환경설정
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
os.environ['HF_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !pip install -U -q langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.2 MB/s eta 0:00:00


In [ ]:
from IPython.display import Markdown
from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI(model='gemini-pro')
result = llm.invoke('현금 달러투자에 대해서 설명해줘')
Markdown(result.content)

In [ ]:
!pip install -U -q langchain tiktoken pypdf sentence_transformers chromadb

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

SOURCE_folder = "/content/drive/MyDrive/data/brand"

loader = DirectoryLoader(SOURCE_folder, glob='**/*.pdf', show_progress=True, loader_cls=PyPDFLoader)
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
texts = text_splitter.split_documents(pages)

# for p in pages[0:10]:
#     with open('/content/drive/MyDrive/foo.txt', 'a') as f:
#         f.write(p.page_content)

!head -n 10 '/content/drive/MyDrive/foo.txt'

In [16]:
DB_URL = '/content/drive/MyDrive/vector_chrm_gemini/'
# texts = tk_chunks

In [19]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

query = '네이밍의 본질이 무엇인가'

# ko_embed = HuggingFaceEmbeddings(
#     model_name="jhgan/ko-sbert-nli",
#     model_kwargs={'device' : 'cpu'},
#     encode_kwargs={'normalize_embeddings':True}
#     )
# # vector_stores = Chroma.from_documents(texts, hug_embed)
# vector_stores = Chroma.from_documents(
#     texts,
#     ko_embed,
#     persist_directory = DB_URL
# )
# print('hug_embed : Document Stored!!')

load_vector_store = Chroma(
    persist_directory = DB_URL,
    embedding_function = ko_embed,
    )

hug_embed : Document Stored!!


In [ ]:
retriever = load_vector_store.as_retriever(
    search_type='mmr',
    search_kwargs = {'k':3, 'fetch_k':10}
)

retriever.get_relevant_documents(query)

In [23]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableMap

template='''Answer the question as based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
gemini_chat = ChatGoogleGenerativeAI(model='gemini-pro', temperature=0)

chain = RunnableMap({
    "context" : lambda x : retriever.get_relevant_documents(x['question']),
    'question' : lambda x: x['question']
}) | prompt | gemini_chat

Markdown(chain.invoke({'question' : query }).content)